<h1> Power Digit Sum </h1> 

$2^{15} = 32768$ and the sum of the digits is $3+2+7+6+8 = 26$.

What is the sum of the digits of the number $2^{1000}$?

---


In [2]:
/// This is nCk assuming nC(k-1) is known
/// Since nC0 and nCn is always 1, this can always be used
fn binomial_coeffs(n: f64, k:f64, previous: f64) -> usize {
//     ((((n+1-k) as f64)/(k as f64))*(previous as f64)) as usize
//     let frac = ((n as f64) + 1.0 - (k as f64))/(k as f64);
//     println!("{}", frac);
//     (frac * previous as f64) as usize
    let frac = (n+1.0 -k)/k;
    (frac * previous).round() as usize
}

<h3> Attempt 1 </h3>

From Binomial theorem, I can express, 
\begin{equation}
    (x + y)^n = \sum_{k=0}^n {n \choose k} x^{n-k} y^k = \sum_{k=0}^n {n \choose k} x^{k} y^{n-k}
    \label{eq:1}
\end{equation}
What I want is the sum of digits, therefore, it would be desireable if I express a number in terms of 
\begin{equation*}
    \sum_{i=0}^n a_i x^i
\end{equation*}
where $a_i$ are coefficients and $x=10$.

From Equation (\ref{eq:1}), I can express it as, 
\begin{align*}
    (x - 8)^n =& \sum_{k=0}^n {n \choose k} x^{k} (-8)^{n-k} \\
    (10-8)^n = 2^n =& \sum_{k=0}^n {n \choose k}  (-8)^{n-k} 10^{k} \\
    =& \sum_{k=0}^n {n \choose k}  (-1)^{n-k} 2^{3(n-k)} 10^{k}
\end{align*}

If I had done this expansion and thought a little harder, I should've realised that this will result in numbers growing faster than if I calculated $2^n$ directly. 

In [3]:
:dep num = "*"
extern crate num;
// use num;

fn poly_coeffs(power:u32) -> Vec<i32> {
    let powusize: usize = power as usize;
    let maxlen: usize = powusize + 1;
    let mut out: Vec<i32> = vec![0 as i32; maxlen];
    let mut prev_binom_coeff: usize = 1;
    
    for i in 0..maxlen {
        out[i] = num::pow(-8i32, powusize-i)*(prev_binom_coeff as i32);
//         println!("{}, {}, {}", prev_binom_coeff, out[i], i);
        prev_binom_coeff = binomial_coeffs(powusize as f64, (i+1) as f64, prev_binom_coeff as f64); 
//         println!("{}, {}", prev_binom_coeff, i+1);
    }
    
    out
}

In [4]:
let sol = poly_coeffs(6);
println!("{:?}", sol);

[262144, -196608, 61440, -10240, 960, -48, 1]


<h3> Attempt 2 </h3>

Find 
\begin{equation}
    \label{eq:2}
    2^n = \sum_{k=0}^n {n \choose k}
\end{equation}
directly. 

This doesn't work as ${n \choose k}$ gets very large

In [5]:
fn binom_coeffs_half(power: u32) -> Vec<usize> {
    
    let mut midpoint: usize = (power/2 + (power)% 2) as usize;
    let mut is_symmetric: bool = true;
    
    if power%2 == 0 {
        midpoint += 1;
        is_symmetric = false;
    }
    
    println!("{} {} {}", power/2, (power)% 2, midpoint);
    
    let mut out: Vec<usize> = vec![0 as usize; midpoint]; 
    let mut prev_binom_coeff: usize = 1;
    out[0] = 2;
    
    for i in 1..midpoint {
        if is_symmetric || (!is_symmetric && i != midpoint -1) {
            out[i] = binomial_coeffs(power as f64, i as f64, prev_binom_coeff as f64); 
            prev_binom_coeff = out[i];
            out[i] = 2*out[i];
        } else {
            out[i] = binomial_coeffs(power as f64, i as f64, prev_binom_coeff as f64); 
            prev_binom_coeff = out[i];
        }
    }
    
    out
}

In [6]:
let sol = binom_coeffs_half(15);
println!("{:?}", sol);
let temp: usize = sol.iter().sum();
println!("{}", temp);

7 1 8
[2, 30, 210, 910, 2730, 6006, 10010, 12870]
32768


In [7]:
let k = (15.0*2.0_f64.log(10.0)) as usize +2;

// This is dumb and inefficient
for i in 1..k {
    let digit: f64 = sol.iter().map(|x| ((*x as f64)%10.0_f64.powi(i as i32))/10.0_f64.powi(i as i32)).sum();
    println!("{}", ((digit - digit.floor())*10.0) as usize );
};

8
6
7
2
3


<h3> Attempt 3 </h3>

The Vandermonde identity is 
\begin{equation}
    {n+m \choose r} = \sum_{k=0}^r {m \choose k} {n \choose r-k}
\end{equation}
To make it in the same form as Equation (2),
\begin{equation}
    \sum_{r=0}^{n+m} {n+m \choose r} = \sum_{r=0}^{n+m} \sum_{k=0}^r {m \choose k} {n \choose r-k} = 2^{n+m}
\end{equation}
A generalisation of the Vandermonde identity is, 
\begin{equation}
    \sum_{r=0}^{n_1 + n_2 + \ldots + n_p} {n_1 + n_2 + \ldots + n_p \choose r} = \sum_{r=0}^{n_1 + n_2 + \ldots + n_p} \sum_{k_1 + k_2 + \ldots + k_p = r} {n_1 \choose k_1} {n_2 \choose k_2} \ldots {n_p \choose k_p} = 2^{n_1 + n_2 + \ldots + n_p}
\end{equation}
Therefore, by storing the inner summation in a single entry of a vector, the individual terms to make up the final summation are known. 

Since $2^n$ can now be expressed as a summation that doesn't result in overflow, the property of sum of digits ($sd$) can be used. This property is given by, 
\begin{equation}
    sd(N + M) = sd(N) + sd(M) - 9\times q
\end{equation}
where $q$ is the carry over from the addition of $N + M$. This can be applied recursively to find the overall sum of digits.


In [8]:
fn multiplicative_binom_coefficient(n: u32, k: u32) -> u32 {
    let mut out: f64 = 1.0;
    let nfloat = n as f64;
    
    if (k == n) || (k == 0) {return out as u32}
    
    for i in 1..k+1 { 
        let ifloat = i as f64;
        out *= (nfloat + 1.0 - ifloat )/ (ifloat);
    }
//     println!("{}", out);
    out.round() as u32
}

In [65]:
fn sum_of_digits(mut input_num: u32, mut prev_sum: u32) -> u32 {
    if input_num/10 == 0{
        prev_sum += input_num % 10;
        return prev_sum
    } else {
        prev_sum += input_num % 10;
        return sum_of_digits(input_num/10, prev_sum)
    }
}

fn num_digits(exponent: u32) -> u32 {
   ((exponent as f64)*2.0_f64.log10()).round() as u32
}

fn num_digits_f(exponent: u32) -> f64 {
   (exponent as f64)*2.0_f64.log10()
}

In [110]:
fn addition_carry_over(mut x: u32, mut y: u32) -> u32{
    let mut carry: u32 = 0; 
    while x != 0 || y != 0 {
        let digit_x = x % 10;
        let digit_y = y % 10;
        if digit_x + digit_y > 9 {carry += 1};
        x /= 10;
        y /= 10;
    }
    
    return carry 
}

fn multiply_carry_over(mut x: u32, mut y: u32) -> u32{
    let mut carry: u32 = 0; 
    while x != 0 || y != 0 {
        let digit_x = x % 10;
        let digit_y = y % 10;
        let mult = digit_x * digit_y;
        if mult > 9 {carry += mult/9};
        x /= 10;
        y /= 10;
    }
    
    return carry 
}

In [11]:
extern crate itertools;
use itertools::Itertools;

fn get_unique_combinations(sum: u32, max_n_value: u32, p: u32) -> Vec<Vec<u32>> {
    
    let mut out: Vec<Vec<u32>> = Vec::new();
    let it = (0..max_n_value+1).combinations_with_replacement(p as usize); 
    
    for item in it {
        let combination_sum: u32 = item.iter().sum();
//         println!("{:?}", item);
//         println!("{}", combination_sum);
        
        if combination_sum == sum {
            out.push(item.clone());
            for perm in item.clone().into_iter().permutations(p as usize).unique() {
                if &perm != &item {out.push(perm.clone())}
            }
            
        }
    }
//     if i > sum {return out}
    out
}

In [12]:
fn set_k_value(r_val: u32, curr_k_sum: u32, curr_n: u32) -> u32 {
    if r_val > curr_k_sum + curr_n {
        return curr_n
    } else {
        return r_val - curr_k_sum
    }
}

In [13]:
fn addition_sum_of_digits(first_num: u32, second_num: u32) -> u32 {
    sum_of_digits(first_num, 0) + sum_of_digits(second_num, 0) - 9*addition_carry_over(first_num, second_num)
}

In [54]:
use std::cmp;

fn add_sd(sd1: u32, sd2: u32, exponent:u32) -> u32 {
//     println!("sd1 + sd2 = {}, (sd1 + sd2)/9 = {}", sd1 + sd2, (sd1 + sd2)/9);
    let sum: u32 = sd1 + sd2;
    let div_nine: u32 = sum/9;
    if sum % 9 == 0 {
        return 0
    } else {
        let order = num_digits(exponent);
        if div_nine > order {
            return sum % 9 + (cmp::max(div_nine, order))*9
//             return sum % 9 + (div_nine - order)*9
        }
            return sum % 9
    }
}

In [34]:
fn final_sum_of_digits(all_digit_sums: Vec<u32>, exponent: u32) -> u32 {
    let tot_num_summation_terms = all_digit_sums.len();
    let num_terms = tot_num_summation_terms/2 + tot_num_summation_terms % 2;
    
    let is_odd: bool = if tot_num_summation_terms % 2 == 0 {false} else {true};
    
    println!("{:?}, {}", all_digit_sums, all_digit_sums.iter().sum::<u32>());
    
    if num_terms == 1 {
        let sd1 = all_digit_sums[0];
        let sd2 = all_digit_sums[1];
        println!("sd1 + sd2 = {}, (sd1 + sd2)/9 = {}", sd1 + sd2, (sd1 + sd2)/9);
//         let out = (sd1 + sd2)%9;
        let out = add_sd(sd1, sd2, exponent);
        println!("{}", out);
        return out
    } else {
        let mut subset: Vec<u32> = vec![0 as u32; num_terms];
        let mut i: usize = 0;
        
        while i*2 + 1 < tot_num_summation_terms {
            let sd1 = all_digit_sums[i*2];
            let sd2 = all_digit_sums[i*2+1];
            println!("sd1 + sd2 = {}, (sd1 + sd2)/9 = {}", sd1 + sd2, (sd1 + sd2)/9);
            subset[i] = add_sd(all_digit_sums[i*2],  all_digit_sums[i*2+1], exponent);
            i += 1;
        }
        
//         println!("{:?} ", subset);
        
        if is_odd {
            subset[num_terms - 1] = all_digit_sums[tot_num_summation_terms - 1];
        }
        
        return final_sum_of_digits(subset, exponent)
    }
        
}

In [16]:
fn summation_terms(k_val_permutations: Vec<Vec<u32>>, p: usize, n: u32) -> Vec<u32>{
    let mut j: usize = 0;
    let mut all_products: Vec<u32> = vec![0 as u32; k_val_permutations.len()];

    for k_set in k_val_permutations.iter() {
        let mut product = 1;
        for i in 0..p {
            let this_k: u32 = k_set[i];
            if this_k != 0 || this_k != n {
                product *= multiplicative_binom_coefficient(n, this_k);
            }
        }
        all_products[j] = product;
        j += 1;
    }
    
    all_products
}


In [17]:
fn digit_sum_of_summation_terms(summation_terms: Vec<u32>, is_first_iteration: bool, exponent: u32) -> u32 {
    let tot_num_summation_terms = summation_terms.len();
    let num_terms = tot_num_summation_terms/2 + tot_num_summation_terms % 2;
    
    let is_odd: bool = if tot_num_summation_terms % 2 == 0 {false} else {true};
    
//     println!("{:?}, {}", summation_terms, summation_terms.iter().sum::<u32>());
    
    if num_terms == 1 {
        if is_first_iteration {
            let out= addition_sum_of_digits(summation_terms[0], summation_terms[1]);
//             println!("{}", out);
            return out
        } else {
            let out = add_sd(summation_terms[0], summation_terms[1], exponent);
//             println!("{}", out);
            return out
        }
    } else {
        let mut subset: Vec<u32> = vec![0 as u32; num_terms];
        let mut i: usize = 0;
        
        while i*2 + 1 < tot_num_summation_terms {
//             println!("i = {}", i);
            if is_first_iteration {
                subset[i] = addition_sum_of_digits(summation_terms[i*2], summation_terms[i*2+1]);
            } else {
                subset[i] = add_sd(summation_terms[i*2],  summation_terms[i*2+1], exponent);
            }
            i += 1;
        }
        
//         println!("{:?} ", subset);
        
        if is_odd && is_first_iteration {
            subset[num_terms - 1] = sum_of_digits(summation_terms[tot_num_summation_terms - 1], 0);
        } else if is_odd {
            subset[num_terms - 1] = summation_terms[tot_num_summation_terms - 1];
        }
        
        return digit_sum_of_summation_terms(subset, false, exponent)
    }
        
}

In [18]:
fn inner_summation(n_val: u32, p: u32, r: u32) -> u32 {
    
    let k_values = get_unique_combinations(r, n_val, p);
//     println!("{:?}", k_values);
    
    let all_products = summation_terms(k_values, p as usize, n_val);
//     println!("prods {:?}", all_products);
    
    digit_sum_of_summation_terms(all_products, true, r)
}

In [19]:
fn outer_summation(exponent: u32, max_n: u32) -> Vec<u32> {
    let quotient = exponent/max_n;
    if exponent % max_n != 0 {panic!("Exponent must be a multiple of n")}
    
    let mut inner_values: Vec<u32> = vec![0 as u32; (exponent+1) as usize];
    inner_values[0] = 1;
    inner_values[exponent as usize] = 1;
    
    for i in 1..exponent {
        inner_values[i as usize] = inner_summation(max_n, quotient, i);
    }
    
    inner_values
//     digit_sum_of_summation_terms(inner_values)
}

In [20]:
fn digit_sum(input_val: u32) -> u32 {
    return input_val % 9
}

fn power_2_digit_sum(exponent: u32) -> u32 {
    let m = exponent % 6;
    return match m {
        0 => 1,
        1 => 2,
        2 => 4,
        3 => 8,
        4 => 7,
        5 => 5,
        6 => 1,
        // This brach is not actually possible 
        _ => panic!(),
    }
}



In [56]:
// let sol = sum_of_digits(1024, 0);
let pow: u32 = 20;
let sol = outer_summation(pow, 10);
// let solsum1: u32= sol.iter().sum();
let solsum1: u32= digit_sum_of_summation_terms(sol.clone(), false, pow );
let solsum: u32= final_sum_of_digits(sol.clone(), pow);
println!("{:?}, sd = {}, ds(calc) = {}, ds(truth) = {}, mlg2 = {}", sol, solsum, solsum1, power_2_digit_sum(pow), num_digits(pow));

[1, 2, 1, 6, 3, 6, 6, 3, 6, 2, 4, 2, 6, 3, 6, 6, 3, 6, 1, 2, 1], 76
sd1 + sd2 = 3, (sd1 + sd2)/9 = 0
sd1 + sd2 = 7, (sd1 + sd2)/9 = 0
sd1 + sd2 = 9, (sd1 + sd2)/9 = 1
sd1 + sd2 = 9, (sd1 + sd2)/9 = 1
sd1 + sd2 = 8, (sd1 + sd2)/9 = 0
sd1 + sd2 = 6, (sd1 + sd2)/9 = 0
sd1 + sd2 = 9, (sd1 + sd2)/9 = 1
sd1 + sd2 = 12, (sd1 + sd2)/9 = 1
sd1 + sd2 = 9, (sd1 + sd2)/9 = 1
sd1 + sd2 = 3, (sd1 + sd2)/9 = 0
[3, 7, 0, 0, 8, 6, 0, 3, 0, 3, 1], 31
sd1 + sd2 = 10, (sd1 + sd2)/9 = 1
sd1 + sd2 = 0, (sd1 + sd2)/9 = 0
sd1 + sd2 = 14, (sd1 + sd2)/9 = 1
sd1 + sd2 = 3, (sd1 + sd2)/9 = 0
sd1 + sd2 = 3, (sd1 + sd2)/9 = 0
[1, 0, 5, 3, 3, 1], 13
sd1 + sd2 = 1, (sd1 + sd2)/9 = 0
sd1 + sd2 = 8, (sd1 + sd2)/9 = 0
sd1 + sd2 = 4, (sd1 + sd2)/9 = 0
[1, 8, 4], 13
sd1 + sd2 = 9, (sd1 + sd2)/9 = 1
[0, 4], 4
sd1 + sd2 = 4, (sd1 + sd2)/9 = 0
4
[1, 2, 1, 6, 3, 6, 6, 3, 6, 2, 4, 2, 6, 3, 6, 6, 3, 6, 1, 2, 1], sd = 4, ds(calc) = 4, ds(truth) = 4, mlg2 = 6


In [111]:
for i in 0..32 {
    let ds = power_2_digit_sum(i);
    let mlg2 = num_digits_f(i);
    let mlg2_over2 = (mlg2/2.0).round();
//     println!("ds = {}, mlg2 = {} mlg2/2 = {} ", ds, mlg2, mlg2_over2);
//     println!("2^{} sd = {}, ds+floor(mlg2/2)*9 = {}",i, sum_of_digits(2_u32.pow(i), 0), ds +(mlg2_over2 as u32)*9 );
    println!("2^{} ds = {} sd = {}, ds+(i/6)*9 = {}",i, ds, sum_of_digits(2_u32.pow(i), 0), ds + cmp::min(i/6, mlg2_over2 as u32)*9 );
};

let temp: u32 = multiply_carry_over(2_u32.pow(31), 2_u32.pow(31));
println!("{}", temp);

2^0 ds = 1 sd = 1, ds+(i/6)*9 = 1
2^1 ds = 2 sd = 2, ds+(i/6)*9 = 2
2^2 ds = 4 sd = 4, ds+(i/6)*9 = 4
2^3 ds = 8 sd = 8, ds+(i/6)*9 = 8
2^4 ds = 7 sd = 7, ds+(i/6)*9 = 7
2^5 ds = 5 sd = 5, ds+(i/6)*9 = 5
2^6 ds = 1 sd = 10, ds+(i/6)*9 = 10
2^7 ds = 2 sd = 11, ds+(i/6)*9 = 11
2^8 ds = 4 sd = 13, ds+(i/6)*9 = 13
2^9 ds = 8 sd = 8, ds+(i/6)*9 = 17
2^10 ds = 7 sd = 7, ds+(i/6)*9 = 16
2^11 ds = 5 sd = 14, ds+(i/6)*9 = 14
2^12 ds = 1 sd = 19, ds+(i/6)*9 = 19
2^13 ds = 2 sd = 20, ds+(i/6)*9 = 20
2^14 ds = 4 sd = 22, ds+(i/6)*9 = 22
2^15 ds = 8 sd = 26, ds+(i/6)*9 = 26
2^16 ds = 7 sd = 25, ds+(i/6)*9 = 25
2^17 ds = 5 sd = 14, ds+(i/6)*9 = 23
2^18 ds = 1 sd = 19, ds+(i/6)*9 = 28
2^19 ds = 2 sd = 29, ds+(i/6)*9 = 29
2^20 ds = 4 sd = 31, ds+(i/6)*9 = 31
2^21 ds = 8 sd = 26, ds+(i/6)*9 = 35
2^22 ds = 7 sd = 25, ds+(i/6)*9 = 34
2^23 ds = 5 sd = 41, ds+(i/6)*9 = 32
2^24 ds = 1 sd = 37, ds+(i/6)*9 = 37
2^25 ds = 2 sd = 29, ds+(i/6)*9 = 38
2^26 ds = 4 sd = 40, ds+(i/6)*9 = 40
2^27 ds = 8 sd = 35, ds+(

In [22]:
println!("{}", addition_carry_over(25,25));
println!("{}", sum_of_digits(25, 0));
println!("{}", addition_sum_of_digits(25, 25));
let a = 2 + 5 + 6;
let b = 1 + 2 + 8 + 1 + 2 + 8;
println!("{} {} {}", a%9, b%9, a);
// println!("{} {}", (b - b%9)/(b/9) + b%9, b/9);
println!("{} {}", (b/9 - 1)*9 + b%9, b/9);
// println!("{} {}", (b/9)*9 - b/9, (b/9)*9 - b%9);
let c = 1 + 2 + 4;
let d = 5 + 1 + 2 + 5 + 1 + 2;
println!("{} {} {}", c%9, d%9, c);
// println!("{} {}", (d - d%9)/(d/9) + d%9, d/9);
println!("{} {}", (d/9 -1)*9 + d%9, d/9);
// println!("{} {}", (d/9)*9 - d/9, (d/9)*9 - d%9);

1
7
5
4 4 13
13 2
7 7 7
7 1


<h3> Final Attempt </h3> 

LET'S SOMEWHAT BRUTE FORCE THIS THING!

\begin{equation}
    a^m = a^x a^y a^z \qquad \text{where } m = x+y+z
\end{equation}
So if I split $2^{1000}$ and split it into $x2^{31} + 2^{1000 - 32x}$. I can efficiently find $2^x$ using a left shift bit operation. I then need split the numbers into an array containig the individual digits such that I can eventually perform a [linear convlution](https://en.wikipedia.org/wiki/Sch%C3%B6nhage%E2%80%93Strassen_algorithm). The linear convolution results the multiplication without carrying. From there, I can then find the value of the resulting convolution. By exploiting symmetry of the problem, I can probably reduce the computation of this. So I want to partition $m$ such that I only need to compute half (or less) of the problem. Once, I've reached the point where I've computed half other multiplications (so $2^{m/2}$). I can use the property of sum of digits, 
\begin{equation}
    sd(M \times N) = sd(N) \times sd(M) - 9R
\end{equation}
To find $R$, I don't need to do the multiplication of $N\times M$, just their digits to find the carry over.

In [57]:
enum Unsigned {
    U8(u8),
    U16(u16),
    U32(u32),
    U64(u64),
}

fn find_number_of_digits_in_power2(exponent: Unsigned) -> Unsigned {
    if let Unsigned(x)
    ((exponent.unwrap() as f32)*(2.0_f32).log10()).ceil() 
}

Error: type annotations needed

Error: non-primitive cast: `f32` as `T`

In [37]:
fn power_partition(desired_exponent: u16) -> (Vec<u16>, bool) {
    // Bitwise AND to determine if exponent is even
    let is_even: bool = if desired_exponent & 1 == 0 {true} else {false};
    
    // If not even, make it even
    // Therefore, the partitioning is only for half of the problem such that it is symmetric
    let exponent = (desired_exponent - desired_exponent & 1)/2;
    
    let max_partition_exponent: u16 = (32_f32 - 9.0_f32.log10()/ 2.0_f32.log10()).floor() as u16;
    
    let remainder = exponent % 28;
    let quotient = exponent/28;
    
    let mut out: Vec<u16> = vec![max_partition_exponent as u16; quotient as usize];
    out.push(remainder as u16);
    
    (out, is_even)
}

In [36]:
fn number_into_vec_of_digits(exponent: u32) -> Vec<u16> {
    if exponent > 31 {panic!()}
    
    // For 2^m, the number of digits = mlg2
    let num_digits: usize = ((exponent as f32)*(2.0_f32).log10()).ceil() as usize;
    
    // Initialise array to store the digits
    let mut digits: Vec<u16> = vec![0 as u16; num_digits];
    
    // Get number from left shift
    let mut number: u32 = 1_u32 << exponent;
    
    for i in (0..num_digits).rev() {
        digits[i] = (number % 10) as u16;
        number /= 10;
    }
    
    digits
}

In [ ]:
fn multiply_power_less_than_29(input_num_as_digits: Vec<u16>, multiplicative_power: u32) -> Vec<u16> {
    // 
    let num_digits: usize = ((multiplicative_power as f32)*(2.0_f32).log10()).ceil() as usize;
    let n = input_num_as_digits.len();
    
    let numbers_to_add: Vec<u32> = input_num_as_digits.iter().map(|x| (*x as u32) << multiplicative_power).collect();
    
    let mut multiplication_as_digits = 
    
}

In [43]:
println!("{}", (9_u32) << 28 as u32);
println!("{} {}", 500/31, 500 % 31);
println!("{}", (32_f32 - 9.0_f32.log10()/ 2.0_f32.log10()).floor() as u32);
let digitvec = number_into_vec_of_digits(30);
println!("{:?}", digitvec); 

2415919104
16 4
28
[1, 0, 7, 3, 7, 4, 1, 8, 2, 4]
